# Cleaning

In [1]:
pip install sastrawi
pip install emoji
pip install pyLDAvis

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.8/433.8 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 8.4 MB/s eta 0:00:00


In [2]:
# import gdown
import pandas as pd
import warnings
import re
warnings.filterwarnings('ignore')

In [74]:
data_url='https://drive.google.com/uc?id=1-mYYXF3E_aZaoQKoYBexzrONf1AxSWnL'
data_path='data.csv'
kamus_alay_url='https://drive.google.com/uc?id=14M5i-Dd_oFCCJf4avgTEQeZLO5PCKCzy'
kamus_alay_path='kamus_alay.csv'
kamus_singkatan_url='https://drive.google.com/uc?id=1f1aOazCYJXCnhg7Brf5GelE7yE8-QflK'
kamus_singkatan_path='kamus_singkatan.csv'
gdown.download(data_url, data_path, quiet=False)
gdown.download(kamus_alay_url, kamus_alay_path, quiet=False)
gdown.download(kamus_singkatan_url, kamus_singkatan_path, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1-mYYXF3E_aZaoQKoYBexzrONf1AxSWnL
To: /content/data.csv
100%|██████████| 784k/784k [00:00<00:00, 82.6MB/s]
Downloading...
From: https://drive.google.com/uc?id=14M5i-Dd_oFCCJf4avgTEQeZLO5PCKCzy
To: /content/kamus_alay.csv
100%|██████████| 3.09M/3.09M [00:00<00:00, 166MB/s]
Downloading...
From: https://drive.google.com/uc?id=1f1aOazCYJXCnhg7Brf5GelE7yE8-QflK
To: /content/kamus_singkatan.csv
100%|██████████| 18.9k/18.9k [00:00<00:00, 31.7MB/s]


'kamus_singkatan.csv'

In [75]:
df=pd.read_csv("data.csv")

### Slang dan Abreviasi

In [76]:
kamus_alay=pd.read_csv(kamus_alay_path)
kamus_alay=kamus_alay.rename(columns = {'slang' : 'kamus_slang' , 'formal' : 'kamus_perbaikan'})
# Rekonstruksi data sebagai 'dict'
slang_mapping = dict(zip(kamus_alay['kamus_slang'], kamus_alay['kamus_perbaikan']))
kamus_singkatan = pd.read_csv(kamus_singkatan_path, header=None, names=['sebelum_perbaikan', 'setelah_perbaikan'],delimiter=';')
singkatan_mapping=dict(zip(kamus_singkatan['sebelum_perbaikan'],kamus_singkatan['setelah_perbaikan']))

### Stopword, emoji, dan Stemmer Factory

In [78]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import  StopWordRemoverFactory
import emoji
from spacy.lang.id import Indonesian
import string

In [79]:
stopword_factory = StopWordRemoverFactory()
stopwords = stopword_factory.get_stop_words()
# List of words with negation meaning
data = emoji.EMOJI_DATA

# Remove negation words from stopwords
# stopwords = set(stopwords).difference(excluded_stopwords)
nlp = Indonesian()
factory = StemmerFactory()
stemmer = factory.create_stemmer()

In [80]:
def replace_emoji_with_space(text, emoji_data, language='id'):
    for emoji, translations in emoji_data.items():
        if language in translations:
            text = text.replace(emoji, '')
    return text

In [81]:
def process_tweet(tweet) :
  tweet=tweet.lower()
  # link
  tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','',tweet)

  # spesifik
  # tweet = re.sub(r'\[username\]|\[url\]|\[askmf\]|\[sensitive-no\]|\[satu menit kemudian\]|\[seo in ha, love rain\]|\[bb\]|\[c48\]|\[idm\]', '', tweet)

  # emoji
  tweet=replace_emoji_with_space(tweet,data)
  # tweet=tweet.replace(":",' ').replace('_','mask').replace('-','rus').strip()
  # tweet=re.sub(' +', ' ', tweet)

  # tokenisasi
  tokens = tweet.split()

  tweet_tokens = []
  for ele in tokens:
    ele_kamus = kamus_singkatan.get(ele, ele)
    ele_slang = slang_mapping.get(ele_kamus, ele_kamus)
    tweet_tokens.append(ele_slang)

  tweet = ' '.join(tweet_tokens)
  tweet = re.sub('[\s]+', ' ', tweet)
    #Replace #word with word
  tweet = re.sub(r'#([^\s]+)', '', tweet)
  tweet=re.sub(r'\d+', '', tweet)
  tweet = tweet.strip('\'"')
  tweet = tweet.lstrip('\'"')

  tweet = "".join([char for char in tweet if char not in string.punctuation])

  doc = nlp(tweet)

  tokens = [token.text for token in doc]
      # Hapus stopwords dari tokens
  filtered_tokens = [token for token in tokens if token.lower() not in stopwords]
  tweet = ' '.join(filtered_tokens)

  tweet=stemmer.stem(tweet)
  # tweet=tweet.replace('mask',' ').replace('rus','-')


  return tweet

In [82]:
df.head(5)

,conversation_id_str,created_at,favorite_count,full_text,id_str,image_url,in_reply_to_screen_name,lang,location,quote_count,reply_count,retweet_count,tweet_url,user_id_str,username
0,1608975983096528897,Fri Dec 30 23:59:02 +0000 2022,0,Inmendagri itu terbit menyusul pencabutan kebi...,1608975983096528897,NaN,NaN,in,Indonesia,0,0,0,https://twitter.com/tauberitamedia/status/1608...,1520998271161831424,tauberitamedia
1,1608975761289121792,Fri Dec 30 23:58:09 +0000 2022,2,Kemarin kebijakan PPKM dicabut hingga sistem p...,1608975761289121792,NaN,NaN,in,NKRI,0,0,1,https://twitter.com/antaranews/status/16089757...,18071520,antaranews
2,1608975421181431808,Fri Dec 30 23:56:48 +0000 2022,8,Kebijakan yg absurd... Gak ada feel kah bahwa ...,1608975421181431808,NaN,NaN,in,NaN,0,0,2,https://twitter.com/taufik_q/status/1608975421...,175752154,taufik_q
3,1608975087327117313,Fri Dec 30 23:55:28 +0000 2022,26,Presiden Jokowi telah mengumumkan pencabutan k...,1608975087327117313,NaN,NaN,in,NaN,1,7,16,https://twitter.com/detikcom/status/1608975087...,69183155,detikcom
4,1608974724490752002,Fri Dec 30 23:54:02 +0000 2022,0,Presiden Jokowi telah mengumumkan pencabutan k...,1608974724490752002,NaN,NaN,in,Indonesia,0,0,0,https://twitter.com/tauberitamedia/status/1608...,1520998271161831424,tauberitamedia


In [85]:
df['full_text'] = df['full_text'].apply(lambda x: process_tweet(str(x)))

In [86]:
df.to_csv('cleaned.csv')

In [89]:
(df['full_text'])

'inmendagri terbit susul cabut bijak laku batas giat masyarakat ppkm presiden'

# Corpus

### Clean Data ke Corpus

In [2]:
data_clean_url='https://drive.google.com/uc?id=13S8a6HUKDAnJNCmxOS0Yy-pqMZGLCfl7'
data_clean_path='data_clean.csv'
gdown.download(data_clean_url, data_clean_path, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=13S8a6HUKDAnJNCmxOS0Yy-pqMZGLCfl7
To: /content/data_clean.csv
100%|██████████| 591k/591k [00:00<00:00, 117MB/s]


'data_clean.csv'

In [12]:
df = pd.read_csv('cleaned.csv')

In [13]:
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
# from gensim.model import CoherenceModel

In [14]:
def gen_words(texts):
  final = []
  for text in texts:
    new = gensim.utils.simple_preprocess(text, deacc=True)
    final.append(new)
  return final

In [15]:
texts = gen_words(df['full_text'])

In [16]:
print(texts)

[['inmendagri', 'terbit', 'susul', 'cabut', 'bijak', 'laku', 'batas', 'giat', 'masyarakat', 'ppkm', 'presiden'], ['kemarin', 'bijak', 'ppkm', 'cabut', 'hingga', 'sistem', 'milu'], ['bijak', 'absurd', 'enggak', 'feel', 'arti', 'laku', 'diskriminasi', 'rakyat', 'sendiri'], ['presiden', 'jokowi', 'umum', 'cabut', 'bijak', 'ppkm', 'jokowi', 'kata', 'tak', 'batas', 'kerumun', 'gera', 'masyarakat', 'kait', 'corona'], ['presiden', 'jokowi', 'umum', 'cabut', 'bijak', 'ppkm', 'jokowi', 'kata', 'tak', 'batas', 'kerumun', 'gera', 'masyarakat', 'kait', 'corona'], ['indikator', 'perintah', 'tepat', 'pekaes', 'ngebacot', 'asal', 'bunyi'], ['perintah', 'resmi', 'cabut', 'bijak', 'laku', 'batas', 'giat', 'masyarakat', 'ppkm', 'jumat'], ['perintah', 'resmi', 'cabut', 'bijak', 'laku', 'batas', 'giat', 'masyarakat', 'ppkm', 'jumat'], ['bijak', 'car', 'free', 'night', 'terap', 'susul', 'diri', 'tujuh', 'panggung', 'hibur', 'panjang', 'jalan', 'sudirmanthamrin'], ['mau', 'hilang', 'subsidi', 'malu', 'pakai

In [17]:
id2word = corpora.Dictionary(texts)

corpus = []
for text in texts:
  new = id2word.doc2bow(text)
  corpus.append(new)

### LDA Modelling

In [18]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                            id2word=id2word,
                                            num_topics=12,
                                            random_state=42,
                                            update_every=1,
                                            chunksize=100,
                                            alpha='auto'
                                            )

### LDA Visualization

In [19]:
import pyLDAvis
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds='mmds', R=30)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7     -0.283111  0.305012       1        1  25.909333
4     -0.033664  0.433604       2        1  16.740513
0      0.346337  0.071444       3        1  12.177888
2     -0.334239  0.054835       4        1  11.586720
8      0.176382  0.232736       5        1   8.197094
11    -0.132853 -0.104366       6        1   7.041787
3     -0.253649 -0.291729       7        1   4.289100
9     -0.012871 -0.287783       8        1   4.122330
1     -0.033113  0.061322       9        1   3.814325
6      0.219146 -0.300583      10        1   3.058286
10     0.156680 -0.150123      11        1   2.305998
5      0.184955 -0.024369      12        1   0.756627, topic_info=            Term        Freq       Total Category  logprob  loglift
235       dukung  811.000000  811.000000  Default  30.0000  30.0000
515       ganjar  925.000000  925.000000  Default  29.0000  29.0000
20        rakyat  372.000000  372.000000  Default  28.0000  28.0000
6     masyarakat  803.000000  803.000000  Default  27.0000  27.0000
8       presiden  726.000000  726.000000  Default  26.0000  26.0000
...          ...         ...         ...      ...      ...      ...
3969         ala    0.026547   13.523190  Topic12  -9.0112  -1.3492
26          kata    0.026373   25.623127  Topic12  -9.0177  -1.9948
393        alami    0.025863    9.811930  Topic12  -9.0373  -1.0545
8       presiden    0.026722  726.062477  Topic12  -9.0046  -5.3259
25          kait    0.025522   98.451203  Topic12  -9.0506  -3.3737

[485 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
337      11  0.987118        adil
534      10  0.663921      agenda
755       4  0.976660         air
474       6  0.951247        ajar
349       7  0.744443      akibat
...     ...       ...         ...
1634      5  0.984472       wujud
3697      1  0.161250  yogyakarta
3697      2  0.831443  yogyakarta
3697      8  0.005039  yogyakarta
468       3  0.995728         yuk

[513 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[8, 5, 1, 3, 9, 12, 4, 10, 2, 7, 11, 6])

### Dynamic LDA Modelling

In [13]:
ldaseq = gensim.models.ldaseqmodel.LdaSeqModel(corpus=corpus,
                                 id2word=id2word,
                                 time_slice=[503,503,503,503],
                                 num_topics=5)

In [14]:
ldaseq.print_topics(time=0)

[[('bijak', 0.0807023470309289),
  ('presiden', 0.056426989273414115),
  ('ppkm', 0.0557726106888789),
  ('cabut', 0.05222579910524598),
  ('jokowi', 0.04124542748812671),
  ('sama', 0.027317359452982845),
  ('resmi', 0.0247768593933581),
  ('laku', 0.022417827668352472),
  ('batas', 0.0177657090657421),
  ('masyarakat', 0.01763717343477655),
  ('joko', 0.016180879896251988),
  ('widodo', 0.01588136738667351),
  ('indonesia', 0.015273071605699691),
  ('perintah', 0.014700604889319855),
  ('giat', 0.013361709418536014),
  ('ayo', 0.010534233768172389),
  ('covid', 0.010237642010938016),
  ('jumat', 0.009268074560263271),
  ('tanah', 0.009067254992917256),
  ('pandemi', 0.008979727869978619)],
 [('bijak', 0.10388065874289787),
  ('indonesia', 0.07945271759979056),
  ('rakyat', 0.07163194241456433),
  ('ganjar', 0.07026398960401689),
  ('presiden', 0.059827504745594035),
  ('baik', 0.03730444322417785),
  ('perintah', 0.036358079094634545),
  ('bangun', 0.028068336994314623),
  ('semua', 

In [15]:
ldaseq.print_topic_times(topic=0) # evolution of 1st topic

[[('bijak', 0.0807023470309289),
  ('presiden', 0.056426989273414115),
  ('ppkm', 0.0557726106888789),
  ('cabut', 0.05222579910524598),
  ('jokowi', 0.04124542748812671),
  ('sama', 0.027317359452982845),
  ('resmi', 0.0247768593933581),
  ('laku', 0.022417827668352472),
  ('batas', 0.0177657090657421),
  ('masyarakat', 0.01763717343477655),
  ('joko', 0.016180879896251988),
  ('widodo', 0.01588136738667351),
  ('indonesia', 0.015273071605699691),
  ('perintah', 0.014700604889319855),
  ('giat', 0.013361709418536014),
  ('ayo', 0.010534233768172389),
  ('covid', 0.010237642010938016),
  ('jumat', 0.009268074560263271),
  ('tanah', 0.009067254992917256),
  ('pandemi', 0.008979727869978619)],
 [('bijak', 0.08122624260114687),
  ('presiden', 0.057269392851891525),
  ('ppkm', 0.05514338888247777),
  ('cabut', 0.04788602569817555),
  ('jokowi', 0.04072607108969835),
  ('sama', 0.03083920952510447),
  ('resmi', 0.024239824057235113),
  ('laku', 0.02215773945036919),
  ('batas', 0.0176378983

In [16]:
ldaseq.save("modelseq.model")